In [88]:
import pandas as pd
import nltk

In [89]:
newzy = pd.read_csv('../FinalProject/newzy.csv', sep='|', index_col='doc_id')

/Users/matthewthomas/anaconda3/envs/DS5001/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [90]:
OHCO = ['doc_source','doc_id', 'sent_num', 'token_num']

In [4]:
newzy.groupby('doc_source').count()

,doc_title,doc_content,doc_date,doc_url
doc_source,,,,
Breitbart,10012,10011,10012,10012
CNN,38011,36184,38011,38011
Daily Kos,12310,12307,12310,12310
Drudge Report,77063,77063,77063,77063
Fox,83415,58077,83417,83417
Google News,64079,64079,64079,64078
Guardian,12392,12392,12392,12392
NPR,30755,24708,30755,30755
New York Times,21239,21145,21240,21240


## Organize and Tokenize PowerLine Articles

In [91]:
powerline = newzy[newzy.doc_source.isin(['PowerLine'])]

In [92]:
powerline = powerline.reset_index().set_index(OHCO[:2])
powerline.head(3)

doc_title  \
doc_source doc_id                                                 
PowerLine  48                             These Zins Are a Dusi   
           49      NY Times Editorial Board Hacked by The Onion   
           50                            The line Obama crossed   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 doc_content  \
doc_source doc_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [47]:
powerline.doc_content.str.len().describe()

count     9730.000000
mean      2844.561871
std       1942.034802
min         91.000000
25%       1532.250000
50%       2506.000000
75%       3692.000000
max      40421.000000
Name: doc_content, dtype: float64

In [93]:
#filter out very short articles
powerline = powerline[powerline['doc_content'].str.len() > 300]

In [94]:
len(powerline)

9664

In [95]:
#inspect articles to see what needs to be filtered
powerline.doc_content[:20]

doc_source  doc_id
PowerLine   48                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [96]:
#get rid of the jquery stuff
powerline['doc_content'] = powerline['doc_content'].str.replace(r'if\(typeof\(jQuery.*', '')

In [52]:
powerline.doc_content[:1]

doc_source  doc_id
PowerLine   48        (Steven Hayward) I’ve been falling behind on many regular features these days, among them, the Power Line 100, but also the crucial subject of wine.  So here’s a quick catch up guide. Old timers of Paso Robles wines all know the Dusi name.  The Dusi family’s Zinfandel vineyards go back nearly a century.  For decades, if you wanted to find the best Zinfandels from the region, you wanted to find who bought the Dusi crop for that year.  The Dusi family seldom made their own wine—until now.  And here’s the four-minute storyboard for it.  (Footnote: I’ve met Janell Dusi a couple of times, and always said, “When are you guys going to start keeping your own grapes for yourselves?”  The time has come.) 
Name: doc_content, dtype: object

In [98]:
#Take out authors in parantheses. This will delete some other words unfortunately
powerline['doc_content'] = powerline['doc_content'].str.replace(r'\([\w\s]*', '')
powerline.doc_content[:1]


doc_source  doc_id
PowerLine   48        ) I’ve been falling behind on many regular features these days, among them, the Power Line 100, but also the crucial subject of wine.  So here’s a quick catch up guide. Old timers of Paso Robles wines all know the Dusi name.  The Dusi family’s Zinfandel vineyards go back nearly a century.  For decades, if you wanted to find the best Zinfandels from the region, you wanted to find who bought the Dusi crop for that year.  The Dusi family seldom made their own wine—until now.  And here’s the four-minute storyboard for it.  : I’ve met Janell Dusi a couple of times, and always said, “When are you guys going to start keeping your own grapes for yourselves?”  The time has come.) 
Name: doc_content, dtype: object

In [99]:
powerline['doc_content'] = powerline['doc_content'].str.replace(r'\)', '')

In [54]:
#Split up sentences
df = powerline['doc_content'].str.split(r'[.?!;:"\(\)]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sentence'})

In [55]:
df = df[~df['sentence'].str.match(r'^\s*$')] # Remove empty sentences
df.head()

sentence
doc_source doc_id                                                                                                                                       
PowerLine  48     1   I’ve been falling behind on many regular features these days, among them, the Power Line 100, but also the crucial subject of wine
                  2                                                                                                     So here’s a quick catch up guide
                  3                                                                               Old timers of Paso Robles wines all know the Dusi name
                  4                                                                       The Dusi family’s Zinfandel vineyards go back nearly a century
                  5    For decades, if you wanted to find the best Zinfandels from the region, you wanted to find who bought the Dusi crop for that year

In [56]:
df.index.names = OHCO[:3]
df.head()

sentence
doc_source doc_id sent_num                                                                                                                                     
PowerLine  48     1          I’ve been falling behind on many regular features these days, among them, the Power Line 100, but also the crucial subject of wine
                  2                                                                                                            So here’s a quick catch up guide
                  3                                                                                      Old timers of Paso Robles wines all know the Dusi name
                  4                                                                              The Dusi family’s Zinfandel vineyards go back nearly a century
                  5           For decades, if you wanted to find the best Zinfandels from the region, you wanted to find who bought the Dusi crop for that year

In [57]:
#split into tokens
df = df['sentence'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame().rename(columns={0:'token'})

In [58]:
df.index.names = OHCO
df = df[~df['token'].str.match(r'^\s*$')]
df.head()

token
doc_source doc_id sent_num token_num         
PowerLine  48     1        1             I’ve
                           2             been
                           3          falling
                           4           behind
                           5               on

In [59]:
#from nltk.tag import StanfordPOSTagger
#stanford_pos_path = '../StanfordParser/stanford-postagger-2018-10-16/stanford-postagger.jar'

In [60]:
#st = StanfordPOSTagger('../StanfordParser/stanford-postagger-2018-10-16/models/english-bidirectional-distsim.tagger', stanford_pos_path)

In [61]:
def add_pos_to_tokens(tokens, ohco=OHCO[:3], token_col='token'):
    """Adds POS tags to tokens."""
    df = tokens.groupby(ohco).token.apply(lambda x: nltk.pos_tag(x.tolist()))\
        .apply(pd.Series).stack()\
        .to_frame().reset_index()\
        .rename(columns={'level_{}'.format(len(ohco)):'token_id', 0:'pos'})\
        .set_index(ohco + ['token_id'])
    tokens['pos'] = df.pos.apply(lambda x: x[1])
    return tokens 

In [62]:
add_pos_to_tokens(df)

token  pos
doc_source doc_id  sent_num token_num                 
PowerLine  48      1        1                I’ve  VBN
                            2                been  VBG
                            3             falling   IN
                            4              behind   IN
                            5                  on   JJ
...                                           ...  ...
           1026333 27       21             people  VBZ
                            22         represents   DT
                            23                  a   JJ
                            24                new   JJ
                            25                low  NaN

[4490019 rows x 2 columns]

## Tokenize Daily Kos Articles

In [101]:
kos = pd.read_csv('dailykos.csv', index_col = OHCO[:2])

In [102]:
del kos['Unnamed: 0']

In [103]:
kos = kos.drop('doc_content', axis=1).rename(columns={'articles':'doc_content'})
kos

doc_title  \
doc_source doc_id                                                                                                         
Daily Kos  96                                               Obama has done nothing to address income inequality. Right?   
           97                                           Anti-racism is not 'anti-white,' no matter what Republicans say   
           98                                    Abbreviated Pundit Round-up:     Angry does not equal populist edition   
           99                                                                            Sunday Talk: Haters gonna hate   
           100                                Gore on fire: Says humans treating atmosphere 'as if it is an open sewer'   
...                                                                                                                 ...   
           922223   CNN's new Republican hack is ratioed trying to walk back attack on Purple Heart recipient's loyalty   
           922224    California announces investigation into PG&amp;E's use of blackouts while still charging customers   
           922225              Chicago teachers say 0.5% of the schools budget stands in the way of ending their strike   
           922226   How do we engage people who've been sitting on the political sidelines? An organizer breaks it down   
           922227  Formerly incarcerated leaders in criminal justice reform question 2020 candidates on their platforms   

                     doc_date  \
doc_source doc_id               
Daily Kos  96      11/03/2013   
           97      11/03/2013   
           98      11/03/2013   
           99      11/03/2013   
           100     11/03/2013   
...                       ...   
           922223  10/31/2019   
           922224  10/31/2019   
           922225  10/31/2019   
           922226  10/31/2019   
           922227  10/31/2019   

                                                                                                                                                                                 doc_url  \
doc_source doc_id                                                                                                                                                                          
Daily Kos  96                                                      http://feeds.dailykos.com/~r/dailykos/index/~3/AUh-9ykkXtQ/-Obama-has-done-nothing-to-address-income-inequality-Right   
           97                                                   http://feeds.dailykos.com/~r/dailykos/index/~3/lMKTZEE84jA/-Anti-racism-is-not-anti-white-no-matter-what-Republicans-say   
           98                                              http://feeds.dailykos.com/~r/dailykos/index/~3/ru5KmshR7u8/-Abbreviated-Pundit-Round-up-Angry-does-not-equal-populist-edition   
           99                                                                                  http://feeds.dailykos.com/~r/dailykos/index/~3/8EXHUux9qb8/-Sunday-Talk-Haters-gonna-hate   
           100                                        http://feeds.dailykos.com/~r/dailykos/index/~3/20HfrYZMVgA/-Gore-on-fire-says-humans-treating-atmosphere-as-if-it-is-an-open-sewer   
...                                                                                                                                                                                  ...   
           922223   http://feeds.dailykosmedia.com/~r/dailykos/index/~3/HmW9jPTWhgY/-CNN-s-new-Republican-hack-is-ratioed-trying-to-walk-back-attack-on-Purple-Heart-recipient-s-loyalty   
           922224        http://feeds.dailykosmedia.com/~r/dailykos/index/~3/w4CnFLjccpo/-California-announces-investigation-into-PG-E-s-use-of-blackouts-while-still-charging-customers   
           922225               http://feeds.dailykosmedia.com/~r/dailykos/index/~3/KEqAc1c0wAY/-Chicago-teachers-say-0-5-of-the-schools-budget-stands-in-the-way-of-ending-their-strike   
           9

In [104]:
#inspect articles to clean them up
kos.doc_content.tail(10)

doc_source  doc_id
Daily Kos   922218                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [105]:
#Get rid of short articles and bad links
kos = kos[kos.doc_content.str.len() > 500]

In [106]:
kos['doc_content'] = kos.doc_content.str.replace(r'Ã¢ÂÂ', '')
kos['doc_content'] = kos.doc_content.str.replace(r'pic\.twitter.*', '')

/Users/matthewthomas/anaconda3/envs/DS5001/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/matthewthomas/anaconda3/envs/DS5001/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [63]:
df_kos = kos['doc_content'].str.split(r'[.?!;:"\(\)]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sentence'})

df_kos = df_kos[~df_kos['sentence'].str.match(r'^\s*$')]

df_kos = df_kos['sentence'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame().rename(columns={0:'token'})

df_kos.index.names = OHCO
df_kos = df_kos[~df_kos['token'].str.match(r'^\s*$')]
df_kos.head()


token
doc_source doc_id sent_num token_num        
Daily Kos  96     0        0             But
                           1             how
                           2          strong
                           3              is
                           4             the

In [64]:
add_pos_to_tokens(df_kos)

token  pos
doc_source doc_id sent_num token_num              
Daily Kos  96     0        0              But   CC
                           1              how  WRB
                           2           strong   JJ
                           3               is  VBZ
                           4              the   DT
...                                       ...  ...
           922227 43       15             one   NN
                           16             way   CC
                           17              or   DT
                           18         another  NaN
                  44       0                ”   NN

[7402674 rows x 2 columns]

## Tokenize Politico

In [107]:
politico = pd.read_csv('politico.csv', index_col=OHCO[:2])

In [108]:
del politico['Unnamed: 0']

In [109]:
politico.doc_content.head(10)

doc_source         doc_id
Politico Magazine  1542            On Jan. 1, 1943, Robert Gene Baker arrived in Washington at the height of World War II to become a Senate page. Two decades later, this son of a mailman from Pickens, S.C., had become the reigning Washington wheeler-dealer and fixer of his day as secretary to the Senate’s Democratic majoritContinue reading ...
                   1545                          This week, world powers and Iran may well be poised to reach an historic agreement to limit the Islamic Republic’s nuclear program—a diplomatic feat that promises to reorder the Middle East and neutralize a threat that has loomed over the region for a decade. The talks, held iContinue reading ...
                   1547                                President Barack Obama has gamely acknowledged that the HealthCare.gov disaster is “on me,” and surely this is one of the more accurate assertions he’s made related to his signature health care program. But no president could

In [110]:
politico = politico[politico.doc_content.str.len() > 500]

In [67]:
df_politico = politico['doc_content'].str.split(r'[.?!;:"\(\)]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sentence'})

df_politico = df_politico[~df_politico['sentence'].str.match(r'^\s*$')]

df_politico = df_politico['sentence'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame().rename(columns={0:'token'})

df_politico.index.names = OHCO
df_politico = df_politico[~df_politico['token'].str.match(r'^\s*$')]
df_politico.head()

token
doc_source        doc_id  sent_num token_num           
Politico Magazine 1010531 0        0                The
                                   1          candidate
                                   2              moves
                                   3             slowly
                                   4            inching

In [68]:
add_pos_to_tokens(df_politico)

token  pos
doc_source        doc_id  sent_num token_num                
Politico Magazine 1010531 0        0                The   DT
                                   1          candidate   NN
                                   2              moves  VBZ
                                   3             slowly   RB
                                   4            inching  VBG
...                                                 ...  ...
                  1009593 206      4                the  NNS
                                   5              cards   IN
                                   6                 at   DT
                                   7                all  NaN
                          207      0                  ”   NN

[7287455 rows x 2 columns]

In [111]:
LIBRARY = pd.concat([powerline, kos, politico])
LIBRARY

doc_title  \
doc_source        doc_id                                                                                           
PowerLine         48                                                                       These Zins Are a Dusi   
                  49                                                NY Times Editorial Board Hacked by The Onion   
                  50                                                                      The line Obama crossed   
                  51                                                                                    Sham/Wow   
                  52                                           A Quarterback Controversy With a Political Twist?   
...                                                                                                          ...   
Politico Magazine 1009144  Opinion | Impeachment Was Supposed to Protect the 2020 Election. Now, It’s Worse Off.   
                  1009145                                What Will the History Books Say About This Impeachment?   
                  1009454                                          Democracy in Inaction: How Trump Beat the Rap   
                  1009592                         What If Everything You Think You Know About Politics Is Wrong?   
                  1009593                                                          The Trump Lawyer Who Survived   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [69]:
TOKEN = pd.concat([df, df_kos, df_politico])

In [70]:
TOKEN['term_str'] = TOKEN['token'].str.lower().str.replace('[\W_]', '')

In [71]:
VOCAB = TOKEN.term_str.value_counts().to_frame().rename(columns={'index':'term_str', 'term_str':'n'})\
    .sort_index().reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'

In [72]:
VOCAB.head()

,term_str,n
term_id,,
0,,161095
1,0,1124
2,00,382
3,000,10361
4,0000,1


### Add Stopwords

In [73]:
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1

In [74]:
VOCAB['stop'] = VOCAB.term_str.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')

In [77]:
VOCAB

,term_str,n,stop
term_id,,,
0,,161095,0
1,0,1124,0
2,00,382,0
3,000,10361,0
4,0000,1,0
...,...,...,...
173073,ツ,7,0
173074,멍청한,1,0
173075,아기,1,0


### Add Stems

In [75]:
#VOCAB = pd.read_csv('VOCAB.csv')
#VSHORT = pd.read_csv('VOCAB2.csv')

In [76]:
VOCAB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173078 entries, 0 to 173077
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   term_str  173078 non-null  object
 1   n         173078 non-null  int64 
 2   stop      173078 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.0+ MB


In [78]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
VOCAB['p_stem'] = VOCAB.term_str.apply(stemmer.stem)

### Add Term ID to Token Table

In [79]:
TOKEN['term_id'] = TOKEN.term_str.map(VOCAB.reset_index().set_index('term_str').term_id)

In [80]:
TOKEN.head()

token  pos term_str  term_id
doc_source doc_id sent_num token_num                                
PowerLine  48     1        1             I’ve  VBN      ive    81029
                           2             been  VBG     been    17319
                           3          falling   IN  falling    56838
                           4           behind   IN   behind    17571
                           5               on   JJ       on   108804

### Add POS max to Vocab Table

In [81]:
VOCAB['pos_max'] = TOKEN.groupby(['term_id', 'pos']).pos.count().unstack().idxmax(1)

In [82]:
VOCAB.head()

,term_str,n,stop,p_stem,pos_max
term_id,,,,,
0,,161095,0,,NN
1,0,1124,0,0,CD
2,00,382,0,00,CD
3,000,10361,0,000,NNS
4,0000,1,0,0000,CD


### Add Term Rank to Vocab

In [83]:
if 'term_rank' not in VOCAB.columns:
    VOCAB = VOCAB.sort_values('n', ascending=False).reset_index()
    VOCAB.index.name = 'term_rank'
    VOCAB = VOCAB.reset_index()
    VOCAB = VOCAB.set_index('term_id')
    VOCAB['term_rank'] = VOCAB['term_rank'] + 1

In [84]:
VOCAB.head()

,term_rank,term_str,n,stop,p_stem,pos_max
term_id,,,,,,
151894,1,the,1073523,1,the,NN
154183,2,to,542245,1,to,VB
107889,3,of,495302,1,of,DT
10167,4,and,438035,1,and,NNP
4643,5,a,427633,1,a,NN


In [85]:
TOKEN.to_csv('TOKEN.csv')

In [86]:
VOCAB.to_csv('VOCAB.csv')

In [112]:
LIBRARY.to_csv('LIBRARY.csv')